In [ ]:
import json

# JSON 파일 로드
with open("datasets/상거래(쇼핑)_refined.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# refined가 존재하고, utterance가 시스템 문장이 아닌 경우만 저장
pairs = []
for item in data[:1083]:
    if item["refined"]:
        if "#@시스템#사진#" in item["utterance"]:
            continue  # ⛔ 이 문장은 건너뜀
        input_text = item["utterance"].strip().replace("\t", " ")
        target_text = item["refined"].strip().replace("\t", " ")
        pairs.append(f"정제: {input_text}\t{target_text}")

# 파일로 저장
with open("refine_data_t5.txt", "w", encoding="utf-8") as f:
    for line in pairs:
        f.write(line + "\n")

print(f"✅ 총 {len(pairs)}개의 유효 학습 샘플 저장 완료.")

✅ 총 1058개의 유효 학습 샘플 저장 완료.


In [3]:
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration

# ✅ 1. 텍스트 파일을 HuggingFace Dataset으로 불러오기
dataset = load_dataset("text", data_files={"train": "refine_data_t5.txt"})

# ✅ 2. 모델과 토크나이저 로드 (Korean T5)
model_name = "kykim/t5-kor-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

ModuleNotFoundError: No module named 'datasets'

In [ ]:
def preprocess_function(examples, max_input_length=64, max_target_length=64):
    inputs = [line.split("\t")[0] for line in examples["text"]]
    targets = [line.split("\t")[1] for line in examples["text"]]

    model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        padding="max_length",
        truncation=True,
    )

    labels = tokenizer(
        targets,
        max_length=max_target_length,
        padding="max_length",
        truncation=True,
    ).input_ids

    # label에서 pad 토큰은 -100으로 바꿔야 loss에 무시됨
    labels = [[(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels]
    model_inputs["labels"] = labels

    return model_inputs

# ✅ 전처리 적용
tokenized_dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./t5_refiner",
    per_device_train_batch_size=8,
    num_train_epochs=5,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
    report_to="none",
    fp16=False,  # True로 바꾸면 GPU + Colab Pro에서 더 빠름
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    tokenizer=tokenizer,
)


In [ ]:
# ✅ 학습 시작!
trainer.train()

In [ ]:
model.save_pretrained("./t5_refiner")
tokenizer.save_pretrained("./t5_refiner")